# 進入DRRIVER，確認一些function-進入主要處理頁面

In [31]:
import os
import base64, re
from io import BytesIO
from PIL import Image
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
import smtplib
from email.message import EmailMessage
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
import cv2
import numpy
from matplotlib import pyplot as plt
import datetime
from datetime import timedelta
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import sys
b=time.time()
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.support.ui import WebDriverWait#直到True 不斷等待
# from selenium.webdriver.support import expected_conditions as EC #判斷某物件存在，直到時間結束
# import xl2dict



###Date
today = datetime.date.today()
first = datetime.date(day=1, month=today.month, year=today.year)
lastMonthFinal = first - datetime.timedelta(days=1)
lastMonthBegan = datetime.date(day=1,month=lastMonthFinal.month, year=today.year)
yearTaiwan=str(lastMonthFinal.year-1911)
if lastMonthBegan.month>=10:
    lastMonth=str(lastMonthBegan.month)
else:
    lastMonth="0"+str(lastMonthBegan.month)
###Date
    
# chrome_options = Options()
def base64_to_image(base64_str, image_path=None):
    base64_data = re.sub('^data:image/.+;base64,', '', base64_str)
    byte_data = base64.b64decode(base64_data)
    image_data = BytesIO(byte_data)
    img = Image.open(image_data)
    if image_path:
        img.save(image_path)

def save_captcha():
    driver.switch_to.default_content()
    driver.switch_to.frame(1)
    try:
        driver.execute_script('''
        img = document.getElementById("CAPTCHA");
        var imgCanvas = document.createElement("canvas"),
        imgContext = imgCanvas.getContext("2d");
        imgCanvas.width = img.width;
        imgCanvas.height = img.height;
        imgContext.drawImage(img, 0, 0, imgCanvas.width, imgCanvas.height);
        imgInfom = imgCanvas.toDataURL("image/png");
        localStorage.setItem("imgInfo",imgInfom);
        ''')
        img_data = driver.execute_script('return localStorage.getItem("imgInfo")')
        base64_to_image(img_data,"./captchaDir/CAPTCHA.png")
    except Exception as e:
        print(str(e))

        
if not os.path.exists("captchaDir"):
    os.mkdir("captchaDir")
driver_path = os.path.join(os.getcwd(), 'chromedriver')
# driver = webdriver.Chrome(executable_path=driver_path, chrome_options=chrome_options)
driver = webdriver.Chrome(executable_path=driver_path)
driver.get("https://waste.epa.gov.tw/prog/IndexFrame.asp?Func=2")


# email

In [32]:

with open('Email.txt') as f:
    mails = f.read().split('\n')
while "" in mails:
    mails.remove("")
mails = set(mails)

def mailTo(title, mailAdds, message, whoSend='foodSaveAutoApply'):
    msg = MIMEMultipart()
    sender = whoSend
    subject = title
    body = message
    msg['From'] = sender
    msg['To'] = ','.join(mailAdds)
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))
    text = msg.as_string()
    # print text
    # Send the message via our SMTP server
    s = smtplib.SMTP('192.168.2.1', 25)
    s.sendmail(sender, mailAdds, text)
    s.quit()
    
    

# 須申報店家資料-給食安維護用資料

In [33]:
import pandas as pd
import sys
df=pd.read_excel(r"D:\trydata\foodSave\DienData.xlsx", encoding=sys.getfilesystemencoding())
data=df.T.to_dict()
needData=[{"店碼":str(data[i]['門店代碼']),
  "帳":data[i]['管制編號'],
  "密":data[i]['IWR&MS密碼'],
  "報廚餘":data[i]['需申報廚餘'],
  "報廢油":data[i]['需申報廢油']} for i in data]

In [34]:
needData[0]

{'報廚餘': 'V', '報廢油': 'V', '密': 'BB47610094##', '帳': 'B20A7339', '店碼': '11702'}

# 店家申報數值(之後改成資料庫)

In [35]:
##資料庫版本
# from pymongo import MongoClient,UpdateOne
# client=MongoClient('localhost',27017,username='j122085',password='850605')
# db=client.rawData.foodSave
# # data=list(db.find({'資料月份':{"$ne":"06"}},{"_id":False}))
# data=list(db.find({'資料月份':{"$ne":lastMonth}},{"_id":False}))#抓取非這個月得資料

# valueData={}
# for i in data:
#     dataOne={}
#     dataOne[i['店碼']]={}
#     dataOne[i['店碼']]['月初廚餘']=i.get('月底剩餘廚餘',"x") #非該月的月底底的廚餘 就是 這個該月初的廚餘
#     dataOne[i['店碼']]['月初廢油']=i.get('月底剩餘廢油',"x")#非該月的月底底的廢油 就是 這個該月初的廢油
    
#     dataOne[i['店碼']]['該月產出廚餘']=i.get('該月產出廚餘',"x")#門店key進資料庫
#     dataOne[i['店碼']]['該月產出廢油']=i.get('該月產出廢油',"x")#門店key進資料庫
    
#     valueData.update(dataOne)

# valueData

In [36]:
df=pd.read_excel(r"D:\trydata\foodSave\WasteData.xlsx", encoding=sys.getfilesystemencoding())
data=df.T.to_dict()
valueData={str(data[i]['門店代碼']):{
  "月初廚餘":data[i]['廚餘-月初庫存'],
  "該月產出廚餘":data[i]['廚餘-該月產出量'],
  "月初廢油":data[i]['廢油-月初庫存'],
  "該月產出廢油":data[i]['廢油-該月產出量']} for i in data}

In [37]:
valueData

{'11229': {'月初廚餘': 13.2, '月初廢油': 'x', '該月產出廚餘': 0.7, '該月產出廢油': 'x'},
 '11231': {'月初廚餘': 6.6, '月初廢油': 'x', '該月產出廚餘': 0.7, '該月產出廢油': 'x'},
 '11246': {'月初廚餘': 6.6, '月初廢油': 'x', '該月產出廚餘': 0.7, '該月產出廢油': 'x'},
 '11252': {'月初廚餘': 13.2, '月初廢油': 'x', '該月產出廚餘': 0.7, '該月產出廢油': 'x'},
 '11255': {'月初廚餘': 9.6, '月初廢油': 'x', '該月產出廚餘': 0.7, '該月產出廢油': 'x'},
 '11702': {'月初廚餘': 0, '月初廢油': 0, '該月產出廚餘': 2, '該月產出廢油': 0.036},
 '11801': {'月初廚餘': 0, '月初廢油': 0, '該月產出廚餘': 1.07, '該月產出廢油': 0.054},
 '11802': {'月初廚餘': 0, '月初廢油': 0, '該月產出廚餘': 1.1, '該月產出廢油': 0.051},
 '11803': {'月初廚餘': 'x', '月初廢油': 0, '該月產出廚餘': 'x', '該月產出廢油': 0.11},
 '11804': {'月初廚餘': 'x', '月初廢油': 0, '該月產出廚餘': 'x', '該月產出廢油': 0.054}}

# 結合兩邊資料 用來做for loop

In [38]:
for i in needData:
    joinData=valueData[i['店碼']]
    if i['報廚餘']=="V":
        i['月初廚餘']=joinData['月初廚餘']
        i['該月產出廚餘']=joinData['該月產出廚餘']
    if i['報廢油']=="V":
        i['月初廢油']=joinData['月初廢油']
        i['該月產出廢油']=joinData['該月產出廢油']

In [39]:
needData[0]

{'報廚餘': 'V',
 '報廢油': 'V',
 '密': 'BB47610094##',
 '帳': 'B20A7339',
 '店碼': '11702',
 '月初廚餘': 0,
 '月初廢油': 0,
 '該月產出廚餘': 2,
 '該月產出廢油': 0.036}

# 登入(含辨識captcha)-需改成帳密變數--由這裡開始for迴圈

In [40]:
def login(account,password):
    try:
        success=0
        while success==0:
            try:
                driver.switch_to.default_content()
            except:
                pass
            driver.switch_to.frame(1)
            driver.page_source
            accountInput=driver.find_element_by_name("fac_user")
            accountInput.clear()
            accountInput.send_keys(account) #帳號
            pwInput=driver.find_element_by_name("fac_pwd")
            pwInput.clear()
            pwInput.send_keys(password) #密碼

            #J55B6752 1121@PINNADA
            #A45B1625 a!0227581123
            ##########################


            captchaText=""
            while len(captchaText)!=4:
                save_captcha()
                mlp=joblib.load(r'D:\trydata\textConsider\model\captcha.pkl') #captcha辨識模型
                # img=Image.open("./captchaDir/CAPTCHA.png")

                pil_image =Image.open("./captchaDir/CAPTCHA.png").convert("RGB")
                open_cv_image=numpy.array(pil_image)
                plt.imshow(open_cv_image)
                imgray=cv2.cvtColor(open_cv_image,cv2.COLOR_BGR2GRAY)
                ret, thresh=cv2.threshold(imgray,127,255,0)
                image,contours,hierarchy=cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
                cnts=sorted([(c,cv2.boundingRect(c)[0]) for c in contours],key=lambda x:x[1])
                ary=[]
                for (c,_) in cnts:
                    (x,y,w,h)=cv2.boundingRect(c)
                #         print((x,y,w,h))
                    if w>11 and h>=7 and w!=120:
                        if x-2>0:
                            ary.append((x-2,y-2,w+4,h+4))
                        else:
                            ary.append((x,y,w+4,h+4))
                n=0

                if not os.path.exists("./captchaDir/numPic/"):
                    os.mkdir("./captchaDir/numPic/")

                for i in os.listdir("./captchaDir/numPic/"):
                    os.remove("./captchaDir/numPic/"+i)

                for (x,y,w,h) in ary:
                    fig=plt.figure(figsize=(6, 4), dpi=100) #600*400
                    roi=open_cv_image[y:y+h,x:x+w]
                    thresh=roi.copy()
                #     ax=fig.add
                    plt.imshow(thresh)
                    plt.savefig("./captchaDir/numPic/aa_{}.png".format(n),dpi=100)
                    n+=1


                data=[]
                basewidth=50
                fig=plt.figure(figsize=(20,20))
                cnt=0
                for idx,img in enumerate(os.listdir("./captchaDir/numPic/")):
                    pil_image=Image.open("./captchaDir/numPic/{}".format(img)).convert('1')
                    wpercent=(basewidth/float(pil_image.size[0]))
                    hsize=int((float(pil_image.size[1])*float(wpercent)))
                    img=pil_image.resize((basewidth,hsize),Image.ANTIALIAS)
                    data.append([pixel for pixel in iter(img.getdata())])
                scaler=StandardScaler()
                scaler.fit(data)
                data_scaled=scaler.transform(data)
    #             print(mlp.predict(data_scaled))

                captchaText="".join(mlp.predict(data_scaled)).replace("xx","")
                if len(captchaText)!=4:
                    driver.find_element_by_link_text("重新載入圖片").click()
                    time.sleep(0.5)


            print("驗證碼",captchaText)
            captchaInput=driver.find_element_by_id("securityCode")
            captchaInput.send_keys(captchaText)
            driver.find_element_by_name("Action").click()
            time.sleep(0.5)
            for i in range(10):
                try:
                    alert=driver.switch_to_alert()
                    alert.accept()
                    time.sleep(0.15)
                except:
                    time.sleep(0.3)

            try:
                time.sleep(1)
                driver.find_element_by_link_text("越境聯單資料錯誤說明")
                success=1
            except:
                print("登入失敗，重試中")
                time.sleep(0.5)
    except:
        report.append("門店:{}登入失敗,錯誤訊息:{},錯誤位置:{}".format(i['店碼'], e, sys.exc_info()[2].tb_lineno))

# 查詢廢廚餘清運量

In [41]:
def getSellDrown():
    driver.get("https://waste.epa.gov.tw/prog/stastic.asp?EPC=FAC")
    

    time.sleep(0.5)

    yb=Select(driver.find_element_by_name("T3"))
    yb.select_by_value(yearTaiwan)

    mb=Select(driver.find_element_by_name("T4"))
    mb.select_by_value(lastMonth)

    db=Select(driver.find_element_by_name("T5"))
    db.select_by_value("0"+str(lastMonthBegan.day))


    ye=Select(driver.find_element_by_name("T6"))
    ye.select_by_value(yearTaiwan)

    me=Select(driver.find_element_by_name("T7"))
    me.select_by_value(lastMonth)

    de=Select(driver.find_element_by_name("T8"))
    de.select_by_value(str(lastMonthFinal.day))

    driver.find_element_by_name("B1").click()
    time.sleep(0.5)

    for i in driver.find_elements_by_css_selector("table"):
        for row in i.text.split("\n"):
            if "再利用" in row.split(" "):
                DrownVolume=row.split(" ")[-1]
#                 print(row)
                print("廢廚餘清運量",DrownVolume)#
                
    return DrownVolume

# 查詢廢食用油清運量

In [42]:
# driver.find_element_by_xpath("/html/body/center/table/tbody/tr/td/table[2]/tbody/tr[2]/td[2]/img")
def getSellOil():
    driver.get("https://waste.epa.gov.tw/prog/stastic.asp?EPC=OilF")
    time.sleep(0.5)
    yb=Select(driver.find_element_by_name("T3"))
    yb.select_by_value(yearTaiwan)

    mb=Select(driver.find_element_by_name("T4"))
    mb.select_by_value(lastMonth)

    db=Select(driver.find_element_by_name("T5"))
    db.select_by_value("0"+str(lastMonthBegan.day))


    ye=Select(driver.find_element_by_name("T6"))
    ye.select_by_value(yearTaiwan)

    me=Select(driver.find_element_by_name("T7"))
    me.select_by_value(lastMonth)

    de=Select(driver.find_element_by_name("T8"))
    de.select_by_value(str(lastMonthFinal.day))

    driver.find_element_by_name("B1").click()
    time.sleep(0.5)

    for i in driver.find_elements_by_css_selector("table"):
        for row in i.text.split("\n"):
            if "廢食用油遞送聯單" in row.split(" "):
                oilVolume=row.split(" ")[-1]
#                 print(row)
                print("廢油清運量",oilVolume)#
    return oilVolume

# 庫存登記

In [43]:
# i=needData[9]
# print(i)
# acc=i['帳']
# pw=i['密']
# oD=i.get('該月產出廚餘',0)
# oO=i.get('該月產出廢油',0)
# logout()
# login(acc,pw)

In [44]:
def storage(surplusOil=0,surplusDrown=0):
    try:
        driver.get("https://waste.epa.gov.tw/prog/Storage/Storage.asp?FacType=F&Role=F")
        time.sleep(0.5)
        driver.find_element_by_name("app_yy").click()
        driver.find_element_by_xpath('//option[@value="' + str(lastMonthFinal.year) + '"]').click()
        driver.find_element_by_name("app_yy").click()
        driver.find_element_by_name("app_mm").click()
        driver.find_element_by_xpath('//option[@value="' + str(lastMonthBegan.month) + '"]').click()
        driver.find_element_by_name("app_mm").click()
        driver.find_elements_by_name("app_submit")[0].click()
        if "您已完成申報上月份貯存情形，如進行修改" in driver.page_source:
            print("該店已申報庫存")
        else:
            driver.find_element_by_link_text("自動帶入").click()
            x=driver.switch_to_alert()
            x.accept()
            for i in driver.find_elements_by_css_selector("tr"):
                if "R-0106" in i.text:
                    print('月底廚餘申報:',str(surplusDrown))
                    brownInput=i.find_element_by_xpath('//input[@id="App_Qty1"]')
                    brownInput.clear()
                    brownInput.send_keys(str(surplusDrown))
                    i.find_element_by_xpath('//input[@name="do_not_save_it_1"]').click()
                if "R-1702" in i.text:
                    print('月底廢油申報:',str(surplusOil))
                    brownInput=i.find_element_by_xpath('//input[@id="App_Qty2"]')
                    brownInput.clear()
                    brownInput.send_keys(str(surplusOil))
                    i.find_element_by_xpath('//input[@name="do_not_save_it_2"]').click()

            driver.find_element_by_xpath('//input[@value="新增"]').click()
            for i in range(5):
                try:
                    time.sleep(0.1)
                    x=driver.switch_to_alert()
                    x.accept()
                except:
                    time.sleep(0.2)
            driver.find_element_by_xpath('//input[@value="正式申報"]').click()
    except:
        report.append("門店:{}報庫存失敗,錯誤訊息:{},錯誤位置:{}".format(i['店碼'], e, sys.exc_info()[2].tb_lineno))

# 廢油、廚餘產出登記

In [45]:
def oilDrownCeclare(outputDrown,outputOil):
    try:
        driver.get("https://waste.epa.gov.tw/massbal/restaurant.asp")

        driver.find_element_by_name("app_year").click()
    #     today = datetime.date.today()
    #     first = datetime.date(day=1, month=today.month, year=today.year)
    #     lastMonthFinal = first - datetime.timedelta(days=1)
    #     yearTaiwan=str(lastMonthFinal.year-1911)
        driver.find_element_by_xpath('//option[@value="' + yearTaiwan + '"]').click()
        driver.find_element_by_name("app_year").click()

        driver.find_element_by_name("app_month").click()
        now=datetime.datetime.now()
        monthTaiwan=str(now.month-1)
        if len(monthTaiwan)==1:
            monthTaiwan="0"+monthTaiwan
        driver.find_element_by_xpath('//option[@value="' + lastMonth + '"]').click()
        driver.find_element_by_name("app_month").click()

        driver.find_elements_by_name("submit")[0].click()

    #     outputDrown=100 #廚餘
    #     outputOil=200 #廢油
        if outputDrown==0 and outputOil==0:
            try:
                driver.find_element_by_link_text("本月無產出廢棄物(新設事業亦適用)").click()
                alert=driver.switch_to_alert()
                alert.accept()
                time.sleep(0.1)
                print("產出量申報量無")
            except:
                driver.find_element_by_link_text("本月無產出廢棄物(新設事業亦適用)(已申報)").click()
                print("無量、系統顯示之前已申報產出量")
        else:
            try:
                driver.find_element_by_link_text("事業廢棄物產出種類、數量").click()
                time.sleep(0.5)
                #這裡部分可能不一樣************************
                for i in driver.find_elements_by_css_selector("tbody"):
                    for j in i.find_elements_by_css_selector("tr"):
                        if "泥狀" in j.text or "固狀" in j.text:
                            productInput=j.find_element_by_id("app_qty")
                            productInput.clear()
                            productInput.send_keys(str(outputDrown))
                            includeWaterInput=j.find_element_by_id("inwater")
                            includeWaterInput.clear()
                            includeWaterInput.send_keys("0")
                            if str(outputDrown)!="0":
                                j.find_element_by_id("chkWasteItem").click()
                                print("申報廚餘",outputDrown)
                            else:
                                print("廚餘量為0不申報")

                        if "液狀" in j.text:
                            productInput=j.find_element_by_id("app_qty")
                            productInput.clear()
                            productInput.send_keys(str(outputOil))
                            includeWaterInput=j.find_element_by_id("inwater")
                            includeWaterInput.clear()
                            includeWaterInput.send_keys("0")
                            if str(outputOil)!="0":
                                j.find_element_by_id("chkWasteItem").click()
                                print("申報廢油",outputDrown)
                            else:
                                print("廚餘量為0不申報")
                driver.find_element_by_xpath('//input[@value="傳送資料"]').click()
                driver.find_element_by_xpath('//input[@value="正式申報"]').click()
                alert=driver.switch_to_alert()
                alert.accept()
                time.sleep(0.1)
                print("產出量申報完畢")
            except:
                driver.find_element_by_link_text("事業廢棄物產生種類、數量(已申報)").click()
                time.sleep(0.5)
                print("系統顯示之前已申報產出量")
            #這裡部分可能不一樣************************
            # driver.find_element_by_name("B1").click()
        #     alert=driver.switch_to_alert()
        #     alert.accept()
        #     time.sleep(0.1)
    except:
        report.append("門店:{}報產出失敗,錯誤訊息:{},錯誤位置:{}".format(i['店碼'], e, sys.exc_info()[2].tb_lineno))

# 回主頁

In [46]:
def logout():
    x=0
    while x==0:
        driver.get("https://waste.epa.gov.tw/prog/IndexFrame.asp?Func=2")
        driver.switch_to_default_content()
        driver.switch_to_frame(0)
        try:
            driver.find_element_by_link_text("登出申報區").click()
            x=1
        except:
            pass

In [48]:
i

{'報廚餘': 'V',
 '報廢油': 'V',
 '密': 'BB47610094##',
 '帳': 'B20A7339',
 '店碼': '11702',
 '月初廚餘': 0,
 '月初廢油': 0,
 '該月產出廚餘': 2,
 '該月產出廢油': 0.036}

In [49]:
report=[]
report.append("\n共有{}項門店須申報".format(len(needData)))


for i in needData:
    if i.get('資料月份',"")!=lastMonth:
        print(str(i))
        acc=i['帳']
        pw=i['密']
        oD=i.get('該月產出廚餘',0)
        oO=i.get('該月產出廢油',0)
        try:
            login(acc,pw)
            print("門店{}登入成功".format(i['店碼']))

            if i['報廚餘']=="V":
                sellDrown=getSellDrown()
                i["月底剩餘廚餘"]=round(i['月初廚餘']+i['該月產出廚餘']-float(sellDrown),3) #變更成資料庫直接撈(月底廚餘)
                if i["月底剩餘廚餘"]<0:
                    i["月底剩餘廚餘"]=0
                    print("門店{}:廚餘月底負量、請確認資料正確性".format(i['店碼']))
                print("月底廢廚餘:{}".format(i["月底剩餘廚餘"]))


            if i['報廢油']=="V":
                sellOil=getSellOil()
                i["月底剩餘廢油"]=round(i['月初廢油']+i['該月產出廢油']-float(sellOil),3)#變更成資料庫直接撈(月底廢油)
                if i["月底剩餘廢油"]<0:
                    i["月底剩餘廢油"]=0
                    print("門店{}:廢油月底負量、請確認資料正確性".format(i['店碼']))
                print("月底廢油{}".format(i["月底剩餘廢油"]))
                
            storage(i.get("月底剩餘廢油",0),i.get("月底剩餘廚餘",0))#直接報月底手key的廢油廢廚餘量
    #         try:
    #             storage(i.get("月底剩餘廢油",0),i.get("月底剩餘廚餘",0))
    #         except:
    #             report.append("門店:{}報庫存失敗,錯誤訊息:{},錯誤位置:{}".format(i['店碼'], e, sys.exc_info()[2].tb_lineno))


            i['資料月份']=lastMonth
            oilDrownCeclare(oD,oO) #直接用清運+月底=產出
    #         try:
    #             oilDrownCeclare(oD,oO)
    #         except:
    #             report.append("門店:{}報產出失敗,錯誤訊息:{},錯誤位置:{}".format(i['店碼'], e, sys.exc_info()[2].tb_lineno))

            logout()
            print("\n")
        except Exception as e:
            report.append("門店:{}原因不明失敗,錯誤訊息:{},錯誤位置:{}".format(i['店碼'], e, sys.exc_info()[2].tb_lineno))
    else:
        print(i['帳'],"該店資料已為最新")

e=time.time()
report.append("\n申報完成，花費時間{}秒".format(round(e-b)))

mailTo("finish", mails, "\n".join(report), whoSend='foodSaveAutoApply')
    
#############################################################以下不認真
driver.switch_to_default_content()
driver.switch_to_frame(0)
driver.execute_script("""d=document;
                        s=d.createElement('script');
                        s.src='https://anohito.tw/thisUnitIsAFlippinPlatelet/flippin_platelet.js';
                        b=d.getElementsByTagName('body')[0];
                        b.appendChild(s);void(0);""")
driver.switch_to_default_content()
driver.switch_to_frame(1)
driver.execute_script("""d=document;
                        s=d.createElement('script');
                        s.src='https://anohito.tw/thisUnitIsAFlippinPlatelet/flippin_platelet.js';
                        b=d.getElementsByTagName('body')[0];
                        b.appendChild(s);void(0);""")
driver.execute_script("""d=document;
                        s=d.createElement('script');
                        s.src='https://anohito.tw/thisUnitIsAFlippinPlatelet/flippin_platelet.js';
                        b=d.getElementsByTagName('body')[0];
                        b.appendChild(s);void(0);""")

print("跳跳血小板：完成~~~~~~~~~~~~~~~~~~~~~~")


    

{'店碼': '11702', '帳': 'B20A7339', '密': 'BB47610094##', '報廚餘': 'V', '報廢油': 'V', '月初廚餘': 0, '該月產出廚餘': 2, '月初廢油': 0, '該月產出廢油': 0.036}


D:\anaconda\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


驗證碼 NHBC
門店11702登入成功
{'店碼': '11229', '帳': 'H42B4328', '密': 'H42B4328818@', '報廚餘': 'V', '報廢油': nan, '月初廚餘': 13.2, '該月產出廚餘': 0.7}
{'店碼': '11231', '帳': 'H42B4338', '密': 'H42B4338818@', '報廚餘': 'V', '報廢油': nan, '月初廚餘': 6.6, '該月產出廚餘': 0.7}
{'店碼': '11246', '帳': 'H43C5021', '密': 'H43C5021818@', '報廚餘': 'V', '報廢油': nan, '月初廚餘': 6.6, '該月產出廚餘': 0.7}
{'店碼': '11252', '帳': 'H43D1591', '密': 'H43D1591818@', '報廚餘': 'V', '報廢油': nan, '月初廚餘': 13.2, '該月產出廚餘': 0.7}
{'店碼': '11255', '帳': 'H49B2595', '密': 'H49B2595818@', '報廚餘': 'V', '報廢油': nan, '月初廚餘': 9.6, '該月產出廚餘': 0.7}
{'店碼': '11801', '帳': 'A45B2663', '密': '1180100000@c', '報廚餘': 'V', '報廢油': 'V', '月初廚餘': 0, '該月產出廚餘': 1.07, '月初廢油': 0, '該月產出廢油': 0.054}
{'店碼': '11802', '帳': 'A36B8404', '密': '6876D7FB674@', '報廚餘': 'V', '報廢油': 'V', '月初廚餘': 0, '該月產出廚餘': 1.1, '月初廢油': 0, '該月產出廢油': 0.051}
{'店碼': '11803', '帳': 'O17L8651', '密': '083DE3D136D@', '報廚餘': nan, '報廢油': 'V', '月初廢油': 0, '該月產出廢油': 0.11}
{'店碼': '11804', '帳': 'B22A8298', '密': 'FC69F7C582A@', '報廚餘': nan, '報廢油': 'V',

WebDriverException: Message: chrome not reachable
  (Session info: chrome=67.0.3396.99)
  (Driver info: chromedriver=2.35.528161 (5b82f2d2aae0ca24b877009200ced9065a772e73),platform=Windows NT 6.1.7601 SP1 x86_64)


# 完成後更新廢油、廢廚餘庫存及資料月份

In [50]:
from pymongo import MongoClient,UpdateOne
client=MongoClient('localhost',27017,username='j122085',password='850605')
db=client.rawData.foodSave

In [ ]:
ids=[i['店碼'] for i in needData]

In [ ]:
operation=[UpdateOne({"_id":idn},{"$set":data},upsert=True) for idn,data in zip(ids,needData)]

In [ ]:
db.bulk_write(operation,ordered=False)

In [51]:
db.find_one({},{"_id":False})

{'報廚餘': 'V',
 '報廢油': 'V',
 '密': 'BB47610094##',
 '帳': 'B20A7339',
 '店碼': '11702',
 '月初廚餘': 0,
 '月初廢油': 0,
 '月底剩餘廚餘': 0.0,
 '月底剩餘廢油': 0.0,
 '該月產出廚餘': 2,
 '該月產出廢油': 0.036,
 '資料月份': '07'}

In [28]:
df=pd.DataFrame(needData)

In [27]:
writer=pd.ExcelWriter(r"D:\outputXLS\foodSave.xlsx")

In [29]:
df.to_excel(writer)

In [30]:
writer.save()